## Project Kickstarter 
by Vivika Wilde wilde.vivika@gmail.com
   Sebastian Fuhrer fuhrer_sebastian@web.de

## Objective 

In recent years, the range of funding options for projects created by individuals and small companies has expanded considerably. In addition to savings, bank loans, friends & family funding and other traditional options, crowdfunding has become a popular and readily available alternative. 

Kickstarter, founded in 2009, is one particularly well-known and popular crowdfunding platform. It has an all-or-nothing funding model, whereby a project is only funded if it meets its goal amount; otherwise no money is given by backers to a project.
A huge variety of factors contribute to the success or failure of a project — in general, and also on Kickstarter. Some of these are able to be quantified or categorized, which allows for the construction of a model to attempt to predict whether a project will succeed or not. The aim of this project is to construct such a model and also to analyse Kickstarter project data more generally, in order to help potential project creators assess whether or not Kickstarter is a good funding option for them, and what their chances of success are.


## Set up

In [1]:
import glob, os, re
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
%matplotlib inline

In [2]:
data = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "data/*.csv"))))
data = data.reset_index(drop=True)

In [3]:
df = data.copy()

## Variable names and description

* **backers_count** - Number of supporters/investors
* **blurb** - A short description of the product written for promotional purposes
* **category** - Projects have been classified into 16 categories. These categories broadly define the genre a project belongs to. **Subcategory** - Categories are further sub-divided in subcategories to give more details on the project. For instance, the category “Technology” has further been split into subcategories like Gadgets, Web, Apps, Software etc. There are 144 total subcategories.
* **converted_pledged_amount** - Total pledged amount in USD.
* **country** - Country of origin of the project
* **created_at** - Date when Project was created (timestamp)
* **creator** - Information on the Creator including ID, name, etc.
* **currency** - Currency used to support the project (3-letter code)**???**
* **currency_symbol** - Symbol of the currency
* **currency_trailing_code** - Defines whether the currency codes are always shown after the amount, independent of the locale.
* **current_currency** - **???**
* **deadline** - The date before which the goal amount has to be gathered.
* **disable_communication** - **???**
* **friends** - No values given (208922 NaN and 300 empty lists)
* **fx_rate** - **???**
* **goal** - Funding amount the project initially asked for **??? USD ???**
* **id** - Project ID
* **is_backing** - No values given (208922 NaN and 300 empty lists)
* **is_starrable** - Whether the project can be marked as favourite or not **???**
* **is_starred** - Whether the project was marked as favourite or not **???**
* **launched_at** - Launch date of the project (timestamp)
* **location** - Project location
* **name** - Project name
* **permissions** - No values given (208922 NaN and 300 empty lists)
* **photo** - Project photo
* **pledged** - Pledge amount in original currency
* **profile** - **???**
* **slug** - **??? Creator-selected keyword id of the project ???**
* **source_url** - URL to the category the project belongs to
* **spotlight** - Spotlight allows creators to make a home for their project on Kickstarter after they've been successfully funded. Each creator can take control of their page and build a customized, central hub for news, updates, links to finished work, and anything else they want the world to know about their project
* **staff_pick** - Staff picks was a feature that highlighted promising projects on the site to give them a boost by helping them get exposure through the email newsletter and highlighted spots around the site. The old 'Kickstarter Staff Pick' badge.
* **state** - Was the project successful at the end of the day? state is a categorical variable divided into the levels successful, failed, live, cancelled, undefined and suspended. 
* **state_changed_at** - Date the state was changed last (timestamp)
* **static_usd_rate** - Conversion rate used by Kickstarter to calculate usd_pledged
* **urls** - URL to the project's side 
* **usd_pledged** - Pledged amount in USD (conversion made by Kickstarter)
* **usd_type** - **???**

## Data types and missings

In [4]:
df.head(2)

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,315,Babalus Shoes,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",28645,US,1541459205,"{""id"":2094277840,""name"":""Lucy Conroy"",""slug"":""...",USD,$,True,...,babalus-childrens-shoes,https://www.kickstarter.com/discover/categorie...,False,False,live,1548223375,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",28645.0,international
1,47,A colorful Dia de los Muertos themed oracle de...,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",1950,US,1501684093,"{""id"":723886115,""name"":""Lisa Vollrath"",""slug"":...",USD,$,True,...,the-ofrenda-oracle-deck,https://www.kickstarter.com/discover/categorie...,True,False,successful,1504976459,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",1950.0,domestic


In [5]:
df.tail(2)

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
209220,76,Seattle Transmedia & Independent Film Festival...,"{""id"":295,""name"":""Festivals"",""slug"":""film & vi...",5692,US,1425256957,"{""id"":307076473,""name"":""Timothy Vernor"",""is_re...",USD,$,True,...,transmedia-gallery-space-stiff-2015,https://www.kickstarter.com/discover/categorie...,True,False,successful,1429536379,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",5692.0,domestic
209221,44,The @1000TimesYes 2009 Tweet Box is a handmade...,"{""id"":13,""name"":""Journalism"",""slug"":""journalis...",1293,US,1263225900,"{""id"":1718677513,""name"":""Article"",""slug"":""arti...",USD,$,True,...,the-1000timesyes-2009-tweet-box,https://www.kickstarter.com/discover/categorie...,True,True,successful,1266814815,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",1293.0,domestic


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 37 columns):
backers_count               209222 non-null int64
blurb                       209214 non-null object
category                    209222 non-null object
converted_pledged_amount    209222 non-null int64
country                     209222 non-null object
created_at                  209222 non-null int64
creator                     209222 non-null object
currency                    209222 non-null object
currency_symbol             209222 non-null object
currency_trailing_code      209222 non-null bool
current_currency            209222 non-null object
deadline                    209222 non-null int64
disable_communication       209222 non-null bool
friends                     300 non-null object
fx_rate                     209222 non-null float64
goal                        209222 non-null float64
id                          209222 non-null int64
is_backing                  300 

### Missing Data

In [7]:
missing = pd.DataFrame(df.isnull().sum(), columns=['Number'])
missing['Percentage'] = round(missing.Number / df.shape[0] * 100, 1)
missing[missing.Number != 0]

,Number,Percentage
blurb,8,0.0
friends,208922,99.9
is_backing,208922,99.9
is_starred,208922,99.9
location,226,0.1
permissions,208922,99.9
usd_type,480,0.2


For the features 'friends', 'is_backing', 'is_starred' and 'permissions' only .1 percent of the data is given.
Therefore these features are not useable  and will be removed from the set. 


In [8]:
df.drop(['friends', 'permissions', 'is_backing', 'is_starred'], axis=1);

In [9]:
df.backers_count.unique()

array([ 315,   47,  271, ..., 3142, 6586, 1192])

### Backers

In [10]:
df.rename(columns = {'backers_count':'backers'}, inplace = True)

### Category

In [11]:
df.category.unique()[0]

'{"id":266,"name":"Footwear","slug":"fashion/footwear","position":5,"parent_id":9,"color":16752598,"urls":{"web":{"discover":"http://www.kickstarter.com/discover/categories/fashion/footwear"}}}'

In [12]:
df_cat = pd.DataFrame.from_dict([json.loads(x) for x in df.category])
df_cat['urls_web'] = pd.DataFrame.from_dict([x for x in df_cat.urls])
df_cat['urls_web_discover'] = pd.DataFrame.from_dict([x for x in df_cat.urls_web])
df_cat.head(3)

,color,id,name,parent_id,position,slug,urls,urls_web,urls_web_discover
0,16752598,266,Footwear,9.0,5,fashion/footwear,{'web': {'discover': 'http://www.kickstarter.c...,{'discover': 'http://www.kickstarter.com/disco...,http://www.kickstarter.com/discover/categories...
1,51627,273,Playing Cards,12.0,4,games/playing cards,{'web': {'discover': 'http://www.kickstarter.c...,{'discover': 'http://www.kickstarter.com/disco...,http://www.kickstarter.com/discover/categories...
2,10878931,43,Rock,14.0,17,music/rock,{'web': {'discover': 'http://www.kickstarter.c...,{'discover': 'http://www.kickstarter.com/disco...,http://www.kickstarter.com/discover/categories...


In [14]:
df[['category', 'subcategory']] = df_cat.slug.str.split("/",expand=True,)

### Country

In [15]:
df.country.unique()

array(['US', 'GB', 'FR', 'AU', 'NZ', 'ES', 'IT', 'NO', 'NL', 'CA', 'SG',
       'MX', 'SE', 'IE', 'DE', 'BE', 'HK', 'AT', 'JP', 'DK', 'CH', 'LU'],
      dtype=object)

### Currency 

In [16]:
df.groupby(['currency', 'currency_symbol','currency_trailing_code', 'current_currency']).size()

currency  currency_symbol  currency_trailing_code  current_currency
AUD       $                True                    CAD                     11
                                                   GBP                      1
                                                   USD                   4854
CAD       $                True                    AUD                      2
                                                   CAD                     23
                                                   GBP                      2
                                                   USD                   9796
CHF       Fr               False                   CAD                      1
                                                   EUR                      1
                                                   USD                    661
DKK       kr               True                    CAD                      1
                                                   USD                    

In [17]:
df.drop('currency_symbol',axis=1);
df.drop('currency_trailing_code',axis=1);

As the currency symbols are less specific and more ambiguous then the currencies themselves only the currency will 
be used as a feature.

### Dates

In [18]:
df_dates['state_changed'] = pd.DataFrame([datetime.fromtimestamp (x) for x in df['state_changed_at']])
df_dates['launched'] = pd.DataFrame([datetime.fromtimestamp (x) for x in df['launched_at']])
df_dates['created'] = pd.DataFrame([datetime.fromtimestamp (x) for x in df['created_at']])
df_dates['deadline'] = pd.DataFrame([datetime.fromtimestamp (x) for x in df['deadline']])
df_dates['duration'] = df_dates['deadline']-df_dates['created']
df_dates['state'] = df['state']

df_dates.head()

NameError: name 'df_dates' is not defined

### Creator

In [20]:
df.creator.unique()[0]

'{"id":2094277840,"name":"Lucy Conroy","slug":"babalus","is_registered":null,"chosen_currency":null,"avatar":{"thumb":"https://ksr-ugc.imgix.net/assets/023/784/556/6ed11b25c853ec1aef7f4360d0eb59ef_original.jpg?ixlib=rb-1.1.0&w=40&h=40&fit=crop&v=1548222691&auto=format&frame=1&q=92&s=b64463d8ae6195f7aeb62393e2ca2dde","small":"https://ksr-ugc.imgix.net/assets/023/784/556/6ed11b25c853ec1aef7f4360d0eb59ef_original.jpg?ixlib=rb-1.1.0&w=160&h=160&fit=crop&v=1548222691&auto=format&frame=1&q=92&s=00bc518b23a932bd76fb6e21f4eb6834","medium":"https://ksr-ugc.imgix.net/assets/023/784/556/6ed11b25c853ec1aef7f4360d0eb59ef_original.jpg?ixlib=rb-1.1.0&w=160&h=160&fit=crop&v=1548222691&auto=format&frame=1&q=92&s=00bc518b23a932bd76fb6e21f4eb6834"},"urls":{"web":{"user":"https://www.kickstarter.com/profile/babalus"},"api":{"user":"https://api.kickstarter.com/v1/users/2094277840?signature=1552621545.c7a32fed985a78dec253fe61c1acb7a99edbc0af"}}}'

In [23]:
# CODE VON ROB UND MAURI
df['creator_RM'] = [df['creator'][i].split('"')[2] for i in range(len(df.creator))]
df['creator_RM'].head()

# sonst vielleicht über eine search im string nach z.B. "ID" und dann dahinter splitten um fehlede werte zu vermeiden 

0    :2094277840,
1     :723886115,
2     :323849677,
3     :196281496,
4    :1178460181,
Name: creator_RM, dtype: object

In [ ]:
#df_creator = pd.DataFrame([x.replace(",",":") for x in df.creator], columns=['creator'])
#df_creator=df['creator'].str.split(",",expand=True,)
#df_creator.info()
#for col in df_creator:
#    df_creator_col = pd.DataFrame(df_creator[col].str.split(":",expand=True,))
#    df_creator = df.creator.join(df_creator_col)

In [ ]:
#df_creator[0]

In [ ]:
#creator_aux = pd.DataFrame([x.replace("{","") for x in df.creator], columns=['creator'])
#creator_aux.creator = pd.DataFrame([x.replace("}","") for x in creator_aux.creator], columns=['creator'])
#creator_aux.creator[0]
#pd.DataFrame.from_dict(json.loads(re(df.creator[0])))

In [ ]:
#df_creator = pd.DataFrame.from_dict([json.loads(x) for x in df.creator])
#df_cat['urls_web'] = pd.DataFrame.from_dict([x for x in df_cat.urls])
#df_cat['urls_web_discover'] = pd.DataFrame.from_dict([x for x in df_cat.urls_web])
#df_creator.head(3)

In [ ]:
#df_creator = df.creator.dropna().apply(pd.Series)

import ast
D1=str(df['creator'][1])
D1=D1.replace('null', "MISSING VALUE")
D1=D1.split('{')[1].replace(',"avatar":',"")
D1 ="{"+ D1+"}"
D1
D2=ast.literal_eval(D1)


In [ ]:
#df_creator = df['creator'].apply(lambda x: x.split(':')[1])
#df.creator.dropna().apply(pd.Series);

In [ ]:
#df_creator = pd.DataFrame([x.replace(",",":") for x in df.creator], columns=['creator'])
#df_creator=df['creator'].str.split(",",expand=True,)
#df_creator

In [ ]:
#df_creator['creator'].replace(',',':',inplace=True)
#df['new_creator'] = df['creator'].apply(lambda x: x.split(':')[1])
#df_creator.append([df_creator['creator'].str.split(":",expand=True,)], ignore_index =True)
#feature3 = [d.get('slug') for d in df.creator]
#feature3

### FX

In [ ]:
(df['fx_rate'] - df['static_usd_rate']).describe().round(4)
df.query('(fx_rate - static_usd_rate) > 0.05').shape[0]

### Pledged Amount

In [ ]:
(df['usd_pledged'] - df['static_usd_rate']* df['pledged']).describe().round()

In [ ]:
(df['converted_pledged_amount'] - df['usd_pledged']).describe().round()

In [ ]:
df.query('(converted_pledged_amount - usd_pledged)/usd_pledged > 0.05').shape[0]

In [ ]:
df['usd_pledged'] = df[['converted_pledged_amount','usd_pledged']].mean(axis=1);

In [ ]:
df.drop('converted_pledged_amount', axis=1);

As usd_pledged is the product of the rate and the original pledged amount, the later two can be removed as features. By definition, the converted_pledged_amount and usd_pledged are assumed to be identical. Therefore, converted_pledged_amount will be removed from the dataset after usd_pledged is adjusted for the deviations. 

### ID

In [ ]:
df.id.nunique()

### Location

In [ ]:
df.creator.unique()[0]

In [ ]:
df.id.unique()[0]

In [ ]:
df.location.unique()[0]

In [ ]:
df_loc = pd.DataFrame.from_dict([json.loads(str(x)) for x in df.location])
#df_cat['urls_web'] = pd.DataFrame.from_dict([x for x in df_cat.urls])
#df_cat['urls_web_discover'] = pd.DataFrame.from_dict([x for x in df_cat.urls_web])
df_loc.head(3)

### Profile

In [ ]:
df.profile[0]

In [ ]:
df_pro = pd.DataFrame.from_dict([json.loads(x) for x in df.profile])

## Label and Features

In [ ]:
df.state.value_counts()

In [ ]:
df.state.hist()

In [ ]:
df.describe().round(2)

## Model